In [85]:
import scipy.io.idl as idl
import numpy as np
from scipy.interpolate import griddata
import idl_plane_to_structured_vtk as idl_to_vtk
reload(idl_to_vtk)

<module 'idl_plane_to_structured_vtk' from 'idl_plane_to_structured_vtk.pyc'>

# Try to store a single time from Jason's IDL scripts
Store $\vec{B}$, $T$, and $n$.

In [86]:
b_x_measurement_points_timestamp = '115110949362'
b_y_measurement_points_timestamp = '115110949522'
b_z_measurement_points_timestamp = '115110938869'
triple_measurement_points_timestamp = '115112535917'
b_x_measurements_timestamp = '115112539367'
b_y_measurements_timestamp = '115112539384'
b_z_measurements_timestamp = '115112539331'
n_measurements_timestamp = '115112539443'
t_measurements_timestamp = '115112539427'

In [87]:
data_path = '../../cataloging_physics_quantities_in_2D/output/intermediate/'
data_ending = '.sav'
b_x_measurement_points_file = data_path + b_x_measurement_points_timestamp + data_ending
b_y_measurement_points_file = data_path + b_y_measurement_points_timestamp + data_ending
b_z_measurement_points_file = data_path + b_z_measurement_points_timestamp + data_ending
triple_measurement_points_file = data_path + triple_measurement_points_timestamp + data_ending 
b_x_measurements_file = data_path + b_x_measurements_timestamp + data_ending
b_y_measurements_file = data_path + b_y_measurements_timestamp + data_ending
b_z_measurements_file = data_path + b_z_measurements_timestamp + data_ending
n_measurements_file = data_path + n_measurements_timestamp + data_ending
t_measurements_file = data_path + t_measurements_timestamp + data_ending

In [88]:
b_x_measurement_points = idl.readsav(b_x_measurement_points_file)
b_y_measurement_points = idl.readsav(b_y_measurement_points_file)
b_z_measurement_points = idl.readsav(b_z_measurement_points_file)
triple_measurement_points =  idl.readsav(triple_measurement_points_file)
b_x_measurements = idl.readsav(b_x_measurements_file)
b_y_measurements = idl.readsav(b_y_measurements_file)
b_z_measurements = idl.readsav(b_z_measurements_file)
n_measurements = idl.readsav(n_measurements_file)
t_measurements = idl.readsav(t_measurements_file)

In [90]:
for time_point in xrange(21):
    file_name = '../output/2016-01-07/temperature_%i.vts' % time_point
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds([t_measurements])
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_scalar(t_measurements,
                                                  time_point,
                                                  x_min, x_max, y_min, y_max)
    (t_resampled, 
     x_grid, y_grid) = idl_to_vtk.remove_nans(t_resampled, x_grid, y_grid)
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.25)
    t_reshaped = idl_to_vtk.reshape_scalar(t_resampled)
    idl_to_vtk.write_to_structured_grid(file_name, t_reshaped, 't', mesh)

In [91]:
for time_point in xrange(21):
    file_name = '../output/2016-01-07/magnetic_field_offset_%i.vts' % time_point
    vector_dicts = [b_x_measurements, b_y_measurements, b_z_measurements]
    (x_min, x_max,
     y_min, y_max) = idl_to_vtk.determine_sample_bounds(vector_dicts)
    (b_resampled, 
     x_grid, y_grid) = idl_to_vtk.resample_vector(vector_dicts[0], 
                                                  vector_dicts[1],
                                                  vector_dicts[2],
                                                  time_point,
                                                  x_min, x_max, y_min, y_max) 
    (b_resampled, 
     x_grid, y_grid) = idl_to_vtk.remove_nans_vector(b_resampled, x_grid, y_grid)
    mesh = idl_to_vtk.prepare_mesh(x_grid, y_grid, 0.3)
    b_vector = idl_to_vtk.reshape_vector(b_resampled[0], b_resampled[1], b_resampled[2])
    idl_to_vtk.write_to_structured_grid(file_name, b_vector, 'B', mesh)

In [92]:
b_resampled[0].shape

(77, 100)

In [93]:
b_resampled[1].shape

(77, 100)

In [94]:
b_vector.shape

(3,)

In [98]:
mesh.shape

(3, 77, 100, 1)

In [95]:
b_vector[0]

array([[[[ -7.05820027e-01],
         [ -1.41604065e-01],
         [  2.15545686e-01],
         ..., 
         [ -1.16983643e-04],
         [ -1.12780313e-04],
         [ -1.06827671e-04]],

        [[ -8.60717258e+00],
         [ -7.04822262e+00],
         [ -6.01845202e+00],
         ..., 
         [ -1.44288923e-04],
         [ -1.44386116e-04],
         [ -1.42131005e-04]],

        [[ -8.10714089e+00],
         [ -7.00263015e+00],
         [ -5.87675909e+00],
         ..., 
         [ -1.53098638e-04],
         [ -1.53880233e-04],
         [ -1.53846270e-04]],

        ..., 
        [[ -8.78777103e-01],
         [  2.18173359e+00],
         [  4.07680834e+00],
         ..., 
         [  8.70274438e-05],
         [ -2.62484354e-06],
         [ -8.90740111e-05]],

        [[ -1.61736515e+00],
         [ -4.22168620e-01],
         [ -3.61090552e-02],
         ..., 
         [  4.08339914e-05],
         [ -3.94329397e-05],
         [ -1.23583228e-04]],

        [[ -4.30254596e-01],
  

In [78]:
b_vector[1]

array([[[[  1.98652783e-04],
         [  7.63005916e-01],
         [ -1.12656325e+00],
         [ -9.67495538e-01],
         [  4.35929616e+00],
         [  2.91970645e-04],
         [ -7.15166947e+00],
         [  8.29344063e+01],
         [  1.65729502e+00],
         [ -2.21597128e+01],
         [ -1.53151597e+01],
         [  1.55077962e+01],
         [ -6.41066962e+00],
         [ -9.12581518e+01],
         [  5.35000083e+00],
         [  8.68677308e+01],
         [ -3.26476352e+00],
         [  8.01971160e+01],
         [  9.77531020e+00],
         [ -9.28832563e+01],
         [ -8.02376414e+01],
         [ -3.92754313e+00],
         [  2.15657380e+01],
         [  2.01582706e+01],
         [  2.46331073e-01]],

        [[  1.02645995e+01],
         [  1.72315896e+01],
         [  2.44201721e+00],
         [ -4.10530032e+00],
         [  3.60800390e+00],
         [  8.63538501e-01],
         [ -4.99866375e+00],
         [  1.56001734e+01],
         [ -1.38323433e+00],
         [ -